In [1]:
!pip install langchain_core
!pip install langchain
!pip install langchain_community
!pip install arxiv
!pip install pypdf
!pip install faiss-gpu
!pip install groq
!pip install fastembed
!pip install wikipedia
!pip install python-dotenv
!pip install langchain-groq
!pip install sentence-transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Using cached nvidia_cusparse_cu12-12.1.0.106-py

In [2]:
!pip install pypdf
!pip install fastembed

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

Taking Content from Wikipedia

In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 2 , max_content_chars_max = 200)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from fastembed import TextEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import DirectoryLoader

In [7]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
api = os.getenv("GROQ_API_KEY")
Model = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=api,model=Model)

In [8]:
def load_pdf(data):
  loader = DirectoryLoader(data,glob="*.pdf",loader_cls = PyPDFLoader)
  documents = loader.load()
  return documents

extracted_data = load_pdf("data/")

In [9]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print(f"Length of the Chunk {len(text_chunks)}")

Length of the Chunk 4445


In [11]:
def embedding_model():
  model_name = "sentence-transformers/all-mpnet-base-v2"
  embedding = HuggingFaceEmbeddings(model_name = model_name)
  return embedding

In [12]:
embeddings = embedding_model()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
sample_embedding = embeddings.embed_query("CDK")
print(f"Length of the sample_embedding{len(sample_embedding)}")
print(f"Example of how the embedding model looks like : {sample_embedding}")

Length of the sample_embedding768
Example of how the embedding model looks like : [-0.00018426278256811202, -0.048676636070013046, -0.007528307382017374, -0.021114999428391457, -0.05875689536333084, 0.04680158570408821, -0.023554950952529907, 0.014673752710223198, -0.029078779742121696, 0.0007294937968254089, 0.05068306252360344, 0.03487849235534668, 0.030592411756515503, 0.050171855837106705, 0.010220727883279324, -0.05152391269803047, -0.01024630106985569, 0.002802246483042836, -0.02841469645500183, -0.018820619210600853, -0.057116519659757614, -0.03917146846652031, -0.04345547407865524, 0.031249217689037323, -0.0036828082520514727, -0.07183065265417099, -0.01749422214925289, 0.005189402028918266, -0.026151884347200394, -0.04057524353265762, 0.06681367009878159, -0.014915616251528263, -0.044715676456689835, 0.053054504096508026, 1.9622966647148132e-06, -0.030229223892092705, -0.006832532584667206, 0.004529524128884077, -0.05685823783278465, 0.0005273165879771113, 0.009841958060860634

In [ ]:
def vector_embedding():
  store = FAISS.from_documents(text_chunks,embeddings)
  embedding = store.as_retriever()
  return embedding
vector_database = vector_embedding()

In [ ]:
prompt_template = """
Hello mister You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not
if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner
context:{context}
question:{question}
"""

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(template = prompt_template , input_variables=["context","query"],chain_type_kwargs={"prompt":prompt_template})

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm = llm,
            chain_type="stuff",
            retriever=vector_database,
            chain_type_kwargs={"prompt":prompt})

In [ ]:
user_input = input("Input Prompt: ")
result = qa_chain({"query": user_input})
final_result = result["result"]

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(vector_database, "qa_tool","Hello mister You are the AI, your name is BrainWave okayy then if user asks the question you need to give the related answer only clearly check the answer whether it is correct or not if it's not correct just inform BrainWave AI doesn't know about this and don't make up an answer and say I don't know about this in a beautiful manner")

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_api = ArxivAPIWrapper()
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 2 , max_content_chars_max = 200)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tools = [wiki,arxiv,retriever_tool]
tools

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
You are a helpful assistant. Your work is to act as a doctor here. If a user asks you any question regarding pain or health issues, you need to provide them with medicine and assist them with instructions on how to take it (e.g., morning 1, evening 1, night 1). You can retrieve information from the PDF, research papers, or even from the web.
Don't provide anything further from this if they ask you about other questions than health, medicine, cure, disease, and the name of medicine you need to answer BrainWave has no idea other than Health Related Questions.
Hello mister, you are the AI, your name is BrainWave. If a user asks the question, you need to give the related answer only. Clearly check the answer whether it is correct or not. If it's not correct, just inform BrainWave AI doesn't know about this and don't make up an answer. Say "I don't know about this" in a beautiful manner.
context:{context}
question:{question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "query"])


In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about AI"})